In [1]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

# Loading Data

In [3]:
ratings = spark.read.load("/FileStore/tables/u.data",format="csv", sep="\t", inferSchema="true", header="false")
ratings = ratings.toDF(*['user_id', 'movie_id', 'rating', 'unix_timestamp'])

In [4]:
ratings.show()

+-------+--------+------+--------------+
user_id|movie_id|rating|unix_timestamp|
+-------+--------+------+--------------+
 196| 242| 3| 881250949|
 186| 302| 3| 891717742|
 22| 377| 1| 878887116|
 244| 51| 2| 880606923|
 166| 346| 1| 886397596|
 298| 474| 4| 884182806|
 115| 265| 2| 881171488|
 253| 465| 5| 891628467|
 305| 451| 3| 886324817|
 6| 86| 3| 883603013|
 62| 257| 2| 879372434|
 286| 1014| 5| 879781125|
 200| 222| 5| 876042340|
 210| 40| 3| 891035994|
 224| 29| 3| 888104457|
 303| 785| 3| 879485318|
 122| 387| 5| 879270459|
 194| 274| 2| 879539794|
 291| 1042| 4| 874834944|
 234| 1184| 2| 892079237|
+-------+--------+------+--------------+
only showing top 20 rows

# 1. Using Spark Functions

The most pysparkish way to create a new column in a pyspark DataFrame is by using the built in functions. This is the most performant programmatical way to create a new column so So this is the first place I go whenever I want to do some column manipulation. We can use `.withcolumn` along with pyspark SQL functions to create a new column. The list of all the functions provided is given [here](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). In essence you can find string functoins, Date functions and math functions already implemented using spark functions.

In [7]:
import pyspark.sql.functions as F

the `F.col` function gives us access to the column.

In [9]:
ratings_with_scale10 = ratings.withColumn("ScaledRating", 2*F.col("rating"))

In [10]:
ratings_with_scale10.show()

+-------+--------+------+--------------+------------+
user_id|movie_id|rating|unix_timestamp|ScaledRating|
+-------+--------+------+--------------+------------+
 196| 242| 3| 881250949| 6|
 186| 302| 3| 891717742| 6|
 22| 377| 1| 878887116| 2|
 244| 51| 2| 880606923| 4|
 166| 346| 1| 886397596| 2|
 298| 474| 4| 884182806| 8|
 115| 265| 2| 881171488| 4|
 253| 465| 5| 891628467| 10|
 305| 451| 3| 886324817| 6|
 6| 86| 3| 883603013| 6|
 62| 257| 2| 879372434| 4|
 286| 1014| 5| 879781125| 10|
 200| 222| 5| 876042340| 10|
 210| 40| 3| 891035994| 6|
 224| 29| 3| 888104457| 6|
 303| 785| 3| 879485318| 6|
 122| 387| 5| 879270459| 10|
 194| 274| 2| 879539794| 4|
 291| 1042| 4| 874834944| 8|
 234| 1184| 2| 892079237| 4|
+-------+--------+------+--------------+------------+
only showing top 20 rows

we can use math functions like `F.exp` function.

In [12]:
ratings_with_exp = ratings.withColumn("expRating", 2*F.exp("rating"))

In [13]:
ratings_with_exp.show()

+-------+--------+------+--------------+------------------+
user_id|movie_id|rating|unix_timestamp| expRating|
+-------+--------+------+--------------+------------------+
 196| 242| 3| 881250949|40.171073846375336|
 186| 302| 3| 891717742|40.171073846375336|
 22| 377| 1| 878887116| 5.43656365691809|
 244| 51| 2| 880606923| 14.7781121978613|
 166| 346| 1| 886397596| 5.43656365691809|
 298| 474| 4| 884182806|109.19630006628847|
 115| 265| 2| 881171488| 14.7781121978613|
 253| 465| 5| 891628467| 296.8263182051532|
 305| 451| 3| 886324817|40.171073846375336|
 6| 86| 3| 883603013|40.171073846375336|
 62| 257| 2| 879372434| 14.7781121978613|
 286| 1014| 5| 879781125| 296.8263182051532|
 200| 222| 5| 876042340| 296.8263182051532|
 210| 40| 3| 891035994|40.171073846375336|
 224| 29| 3| 888104457|40.171073846375336|
 303| 785| 3| 879485318|40.171073846375336|
 122| 387| 5| 879270459| 296.8263182051532|
 194| 274| 2| 879539794| 14.7781121978613|
 291| 1042| 4| 874834944|109.19630006628847|
 234| 1184| 2| 892079237| 14.7781121978613|
+-------+--------+------+--------------+------------------+
only showing top 20 rows

# 2. Spark UDFs

In [15]:
from pyspark.sql.types import *

def somefunc(value):
  if   value < 3: 
      return 'low'
  else:
      return 'high'

#convert to a UDF Function by passing in the function and return type of function
udfsomefunc = F.udf(somefunc, StringType())

ratings_with_high_low = ratings.withColumn("high_low", udfsomefunc("rating"))


In [16]:
ratings_with_high_low.show()

+-------+--------+------+--------------+--------+
user_id|movie_id|rating|unix_timestamp|high_low|
+-------+--------+------+--------------+--------+
 196| 242| 3| 881250949| high|
 186| 302| 3| 891717742| high|
 22| 377| 1| 878887116| low|
 244| 51| 2| 880606923| low|
 166| 346| 1| 886397596| low|
 298| 474| 4| 884182806| high|
 115| 265| 2| 881171488| low|
 253| 465| 5| 891628467| high|
 305| 451| 3| 886324817| high|
 6| 86| 3| 883603013| high|
 62| 257| 2| 879372434| low|
 286| 1014| 5| 879781125| high|
 200| 222| 5| 876042340| high|
 210| 40| 3| 891035994| high|
 224| 29| 3| 888104457| high|
 303| 785| 3| 879485318| high|
 122| 387| 5| 879270459| high|
 194| 274| 2| 879539794| low|
 291| 1042| 4| 874834944| high|
 234| 1184| 2| 892079237| low|
+-------+--------+------+--------------+--------+
only showing top 20 rows

# 3. Using RDD

In [18]:
import math
from pyspark.sql import Row

def rowwise_function(row):
  # convert row to dict:
  row_dict = row.asDict()
  # Add a new key in the dictionary with the new column name and value
  row_dict['Newcol'] = math.exp(row_dict['rating'])
  # convert dict to row:
  newrow = Row(**row_dict)
  # return new row
  return newrow

# convert ratings dataframe to RDD
ratings_rdd = ratings.rdd
# apply our function to RDD
ratings_rdd_new = ratings_rdd.map(lambda row: rowwise_function(row))
# Convert RDD Back to DataFrame
ratings_new_df = sqlContext.createDataFrame(ratings_rdd_new)

ratings_new_df.show()

+------------------+--------+------+--------------+-------+
 Newcol|movie_id|rating|unix_timestamp|user_id|
+------------------+--------+------+--------------+-------+
20.085536923187668| 242| 3| 881250949| 196|
20.085536923187668| 302| 3| 891717742| 186|
 2.718281828459045| 377| 1| 878887116| 22|
 7.38905609893065| 51| 2| 880606923| 244|
 2.718281828459045| 346| 1| 886397596| 166|
54.598150033144236| 474| 4| 884182806| 298|
 7.38905609893065| 265| 2| 881171488| 115|
 148.4131591025766| 465| 5| 891628467| 253|
20.085536923187668| 451| 3| 886324817| 305|
20.085536923187668| 86| 3| 883603013| 6|
 7.38905609893065| 257| 2| 879372434| 62|
 148.4131591025766| 1014| 5| 879781125| 286|
 148.4131591025766| 222| 5| 876042340| 200|
20.085536923187668| 40| 3| 891035994| 210|
20.085536923187668| 29| 3| 888104457| 224|
20.085536923187668| 785| 3| 879485318| 303|
 148.4131591025766| 387| 5| 879270459| 122|
 7.38905609893065| 274| 2| 879539794| 194|
54.598150033144236| 1042| 4| 874834944| 291|
 7.38905609893065| 1184| 2| 892079237| 234|
+------------------+--------+------+--------------+-------+
only showing top 20 rows

# 4. Pandas UDF

In [20]:
# Declare the schema for the output of our function
outSchema = StructType([StructField('user_id',IntegerType(),True),StructField('movie_id',IntegerType(),True),StructField('rating',IntegerType(),True),StructField('unix_timestamp',IntegerType(),True),StructField('normalized_rating',DoubleType(),True)])

# decorate our function with pandas_udf decorator
@F.pandas_udf(outSchema, F.PandasUDFType.GROUPED_MAP)
def subtract_mean(pdf):
    # pdf is a pandas.DataFrame
    v = pdf.rating
    v = v - v.mean()
    pdf['normalized_rating'] =v
    return pdf

rating_groupwise_normalization = ratings.groupby("movie_id").apply(subtract_mean)

rating_groupwise_normalization.show()

+-------+--------+------+--------------+-----------------+
user_id|movie_id|rating|unix_timestamp|normalized_rating|
+-------+--------+------+--------------+-----------------+
 186| 148| 4| 891719774| 0.796875|
 201| 148| 1| 884140751| -2.203125|
 222| 148| 2| 881061164| -1.203125|
 251| 148| 2| 886272547| -1.203125|
 59| 148| 3| 888203175| -0.203125|
 244| 148| 2| 880605071| -1.203125|
 203| 148| 3| 880434755| -0.203125|
 313| 148| 2| 891031979| -1.203125|
 178| 148| 4| 882824325| 0.796875|
 49| 148| 1| 888068195| -2.203125|
 293| 148| 1| 888907015| -2.203125|
 270| 148| 4| 876954062| 0.796875|
 198| 148| 3| 884206401| -0.203125|
 51| 148| 3| 883498623| -0.203125|
 274| 148| 2| 878946133| -1.203125|
 130| 148| 4| 876251127| 0.796875|
 320| 148| 4| 884748708| 0.796875|
 332| 148| 5| 887938486| 1.796875|
 164| 148| 5| 889402203| 1.796875|
 181| 148| 2| 878963204| -1.203125|
+-------+--------+------+--------------+-----------------+
only showing top 20 rows

# 5. Using SQL

In [22]:
ratings.registerTempTable('ratings_table')
newDF = sqlContext.sql('select *, 2*rating as newCol from ratings_table')
newDF.show()

+-------+--------+------+--------------+------+
user_id|movie_id|rating|unix_timestamp|newCol|
+-------+--------+------+--------------+------+
 196| 242| 3| 881250949| 6|
 186| 302| 3| 891717742| 6|
 22| 377| 1| 878887116| 2|
 244| 51| 2| 880606923| 4|
 166| 346| 1| 886397596| 2|
 298| 474| 4| 884182806| 8|
 115| 265| 2| 881171488| 4|
 253| 465| 5| 891628467| 10|
 305| 451| 3| 886324817| 6|
 6| 86| 3| 883603013| 6|
 62| 257| 2| 879372434| 4|
 286| 1014| 5| 879781125| 10|
 200| 222| 5| 876042340| 10|
 210| 40| 3| 891035994| 6|
 224| 29| 3| 888104457| 6|
 303| 785| 3| 879485318| 6|
 122| 387| 5| 879270459| 10|
 194| 274| 2| 879539794| 4|
 291| 1042| 4| 874834944| 8|
 234| 1184| 2| 892079237| 4|
+-------+--------+------+--------------+------+
only showing top 20 rows